# Dataclasses: Introduction to `field()`

## Learning Goals

* How can we control mutability in dataclasses?  
* How do we customize how objects are represented?  
* How can we modify or exclude specific fields from certain behaviors?  
* How does Python compare dataclass objects? 

## Introduction

The `field()` function in Python’s `dataclasses` module allows fine-tuned control over attributes. It is useful for:

- Setting default values correctly (especially for mutable types like lists).
- Hiding fields from `repr` to avoid exposing sensitive data.
- Adding metadata to store additional information about fields.

To use `field`, import it alongside `dataclass`:

```python
    from dataclasses import dataclass, field
```

### Default Values and Mutability

When defining attributes in a class, we must be careful when using **mutable default values** like lists or dictionaries.

> If a __mutable object__ is used as a __default value__, it is __shared across all instances of the class__, leading to possibly unintended behavior.

Consider the case where we define a `courses` attribute with a default empty list. Without `field(default_factory=list)`, every instance of the class would reference the **same** list in memory. This means that modifying the list in one instance would also affect all other instances, even though we expect them to be independent.

In [ ]:
from dataclasses import dataclass # Example without using field

@dataclass
class Student:
    full_name: str
    student_id: str
    courses = [] # Possible Danger

# Creating two students
s1 = Student("Jordan Lee", "s123")
s2 = Student("Taylor Davis", "s456")

# Modifying one student's courses
s1.courses.append("Mathematics")

# Checking the courses of both students
print("s1 courses:", s1.courses)
print("s2 courses:", s2.courses)

As you can see above, both students share the same courses, even if the course was never added to `s2`. This happens because Python assigns the same list object to all instances, rather than creating a new list for each student. By using `field(default_factory=list)`, we ensure that every student gets an independent list that is not shared across instances.

Using `field(default_factory=list)` ensures that each instance gets its **own separate list** instead of sharing a single list. This avoids accidental modifications affecting multiple instances and ensures predictable behavior when working with mutable data types.

We can now see why handling mutable defaults properly is important when designing classes, especially in cases where each object should manage its own independent data.

In [ ]:
from dataclasses import dataclass, field # Importing field along with dataclass

@dataclass
class Student:
    full_name: str
    student_id: str
    courses: list = field(default_factory=list)  # Prevents shared mutable defaults

# Creating two students
s1 = Student("Jordan Lee", "s123")
s2 = Student("Taylor Davis", "s456")

# Modifying one student's courses
s1.courses.append("Mathematics")

# Checking the courses of both students
print("s1 courses:", s1.courses)
print("s2 courses:", s2.courses)

You can see that each of our student object has an individual list now.

### Making Dataclasses Immutable with `frozen=True`

By default, dataclass instances in Python are **mutable**, meaning their attributes can be modified after creation. While this is useful in many cases, there are situations where immutability is preferred. Making a dataclass **immutable** ensures that once an object is created, its attributes cannot be changed, preventing accidental modifications and enforcing data integrity.  

Using `frozen=True` in a `dataclass` makes all fields read-only. This is particularly useful when working with data structures that **should not be modified** after initialization. If an attempt is made to change an attribute, Python will raise an error, ensuring the object remains unchanged throughout its lifecycle.


In [ ]:
from dataclasses import dataclass

@dataclass(frozen=True)
class Student:
    full_name: str
    student_id: str

# Creating a student object
s = Student("Jordan Lee", "s123")

# Attempting to modify an attribute
s.full_name = "Taylor Davis"  # This will raise an error

### Excluding Fields from `repr()` in Dataclasses

When printing an instance, Python generates a string representation (`__repr__`) that shows all attributes. This is useful for debugging, but sometimes you want to hide certain attributes for clarity—such as sensitive data (passwords, API keys) or redundant information—to reduce clutter.

To exclude a field from `repr`, we can use `repr=False` in `field()`. The attribute remains part of the object and can still be accessed normally, but it will not appear when the object is printed.

In the example below, we define a `Student` class where the **university name** is always the same for all students. Since this information is redundant, we exclude it from the representation to keep the output more concise.

In [ ]:
@dataclass
class Student:
    full_name: str
    student_id: str
    university: str = field(default="Academy of Software Arts", repr=False)  # Exclude from repr

# Creating a student object
s = Student("Jordan Lee", "s123")

# Printing the object
print(s)

### Controlling Object Comparisons with `order=True` in Dataclasses

By default, dataclass instances do not support ordering, meaning objects cannot be compared using operators like `<`, `>`, `<=`, or `>=`. However, by setting `order=True`, Python automatically generates ordering methods based on the fields of the dataclass.  

When ordering is enabled, Python compares objects **field by field** in the order they are defined. The first attribute acts as the primary sorting key, and if two objects have the same value for that attribute, the next field is used, and so on.  

To control which attributes are used for ordering, you can **exclude specific fields** by setting `compare=False` in `field()`. This allows you to define fields that are part of the object but should not be considered when comparing instances. Choosing the right attributes for ordering depends on the context.

In the example below, the `borrowed_books` attribute is used as the sorting key, while `full_name` and `student_id` are excluded from comparisons using `compare=False`. This ensures that students are ordered only by the number of books they have borrowed, without other attributes affecting the comparison.

In [ ]:
@dataclass(order=True)
class Student:
    full_name: str = field(compare=False)   # Excluded from ordering
    student_id: str = field(compare=False)  # Excluded from ordering
    borrowed_books: int                     # Primary sorting key

# Creating student objects

s1 = Student("Jordan Lee",   "s123", 3)     # Borrowed 3 books
s2 = Student("Taylor Davis", "s456", 6)     # Borrowed 6 books

# Comparing students
print(s1 > s2)
print(s1 < s2)

### Summary 
Dataclasses provide a structured way to define objects while reducing boilerplate code. With `field()`, we gain more control over how attributes behave, allowing us to refine object representation, enforce constraints, and improve consistency. Understanding these tools helps create more predictable and maintainable code, making object management in Python both clearer and more efficient.
